In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [7]:
roi = ee.FeatureCollection('users/311605001111/region/taihu_vector')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

def add_property(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('LANDSAT_PRODUCT_ID')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
    point = point.set({'row': image.get('WRS_ROW')})
    point = point.set({'path': image.get('WRS_PATH')})
    point = point.set({'cloud': image.get('CLOUD_COVER')})
    return point

landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2013-01-01', '2021-08-29')

filter_images = landsat_images.map(add_property)
print(landsat_images.size().getInfo())
geemap.ee_to_csv(ee.FeatureCollection(filter_images),'taihu_landsat8.csv')

# landsat8

In [ ]:
roi = ee.FeatureCollection('users/311605001111/Yangtze_huangchong')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,6)

In [ ]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}
def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

def add_property(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('LANDSAT_PRODUCT_ID')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
    point = point.set({'row': image.get('WRS_ROW')})
    point = point.set({'path': image.get('WRS_PATH')})
    point = point.set({'cloud': image.get('CLOUD_COVER')})
    return point

In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2019-01-01', '2021-06-03') \
    .filter(ee.Filter.lt('CLOUD_COVER',5)) \
    .map(maskL8sr)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

In [ ]:
# image = landsat_images.mosaic().clip(roi)
# image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
# areas = image_area.reduceRegion(**{
#     'reducer': ee.Reducer.sum(),
#     'geometry': roi.geometry(),
#     'scale': 30,
#     'maxPixels': 1e14
# })
# # print(areas.getInfo())


# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',200))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
# print(dates.getInfo())

In [ ]:
filter_images = sample_image.map(add_property)
print(sample_image.size().getInfo())
geemap.ee_to_csv(ee.FeatureCollection(filter_images),'yangtzeRiver_landsat8.csv')

# sentinel2

In [ ]:
roi = ee.FeatureCollection('users/311605001111/Yangtze_huangchong')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,6)

In [ ]:
rgbVis = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

def add_property(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('id')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
    point = point.set({'satellite name': image.get('SPACECRAFT_NAME')})
    point = point.set({'product_id': image.get('PRODUCT_ID')})
    point = point.set({'mgrs_tile': image.get('MGRS_TILE')})
    point = point.set({'cloud': image.get('CLOUDY_PIXEL_PERCENTAGE')})
    return point

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 15,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

In [ ]:
# 筛选数据（使用日期，云量，研究区），去云
images = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(roi) \
    .filterDate('2019-01-01', '2021-06-03') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
print(images.size().getInfo())
Map.addLayer(images.first(), rgbVis, 'RGB')
# print(images.first().getInfo())

In [ ]:
points = images.map(add_property)

geemap.ee_to_csv(ee.FeatureCollection(points),'yangtzeRiver_sentinel2.csv')